<a href="https://colab.research.google.com/github/BrotherKim/AI599/blob/master/Final_evaluate_drunkendriving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KAIST AI599 법률인공지능

In [1]:
# train variables
TARGET = 'money'
TARGET_SET = 'avail' # all, avail, nonavail
TARGET_START = 1 # 0 if you want zero values, INCLUDE THST VALUE
TARGET_END = 100000000 # INCLUDE THIS VALUE
TARGET_APPROX = 2000000 # '2'개월 또는 '1000000'원
TARGET_OPT = 'class' # class, binary
TRAIN_COLUMN = 'fy' # raw, facts, statutes, yh, fy
MODEL_DIR = '/content/drive/MyDrive/AI599/models'

## 과제 개요

이름: 강여진, 김동규, 김학성, 김형, 여인경

개요
- 컬럼은 한글로 사용
- model은 영대문자 사용
- dataset은 영소문자, 숫자 사용


데이터 준비
- precedent는 주문을 형량으로 바꾼 것(이하 "형"), 양형의 사유(이하 "양"), 그리고 나머지(이하 "나")로 이루어져 있음. 양형의 사유와 나머지를 포함한 것을 "양나"라고 칭함.
- "형"을 벌금만 있는 경우와 징역만 있는 경우, 그리고 집행유예가 있는 징역의 경우를 추출. 각각 "벌", "징", "집"이라 칭함.
- 양형의 사유가 없는 집합에서 "벌"-"나", "징"-"나", "집"-"나" 데이터셋 생성. 이를 a, b, c라고 칭함
- 양형의 사유가 있는 집합에서 "벌"-"나", "징"-"나", "집"-"나", "벌"-"양나", "징"-"양나", "집"-"양나" 데이터셋 생성. 이를 d, e, f, g, h, i라고 칭함.
- a~i까지 데이터 생성 시, 해당하지 않는 라벨에 대해서도 예측하도록 함.


판결문에 대한 형량 예측 모델 학습
- 모델 A, B 준비. A에는 a로 "나"에 대한 "벌" 예측 모델 생성. B에는 b로 "나"에 대한 "징" 예측 모델 생성, C에는 c로 "나"에 대한 "집" 예측 모델 생성.


예측
- A에 d를 eval시켜 "나"에 해당하는 "벌" 예측. 이 결과쌍을 j라고 칭함.
- B에 e를 eval시켜 "나"에 해당하는 "징" 예측. 이 결과쌍을 k라고 칭함.
- C에 f를 eval시켜 "나"에 해당하는 "집" 예측. 이 결과쌍을 l라고 칭함.


양형 사유에 대한 형량 예측 모델 학습
- d, g의 "벌"의 delta 값을 추출. "양"-delta 데이터셋 생성. 이하 m.
- e, h의 "징"의 delta 값을 추출. "징"-delta 데이터셋 생성. 이하 n.
- f, i의 "집"의 delta 값을 추출. "집"-delta 데이터셋 생성. 이하 o.
- 모델 C, D, E 준비. m, n, o에 대해 학습.

## Google drive connect.
connect google drive storage for saving checkpoints.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## lbox_open data 전처리.
lbox_open 데이터를 가져와 형태를 분석합니다.

In [3]:
!pip install -q datasets==2.4.0

## filtering lbox_open data
2019.6.25 이후에 발생한 음주운전 데이터를 추출합니다.

In [4]:
# Precedent 쪼개기 위한 function set

import re

class Precedent:
    def __init__(self, rawPrecedent):
        self.raw = rawPrecedent
        self.__parse__()
        
    def __parse__(self):
        # 주문 - 범죄사실 - 법령의적용 - 양형의이유
        # ms - facts - statuts - yh
        # fy: facts + yh
        retval = re.search(r'주문[\n]*([^\n].+[^\n])[\n]+이유[\n]*([^\n].+[^\n])[\n]*', self.raw, re.S)
        self.ms = retval.group(1) # Main Sentence, 주문
        self.ms = self.ms.strip()

        self.etc = retval.group(2)

        self.yh = ''
        l = self.etc.split('양형의 이유')
        if 1 < len(l):
            self.etc = l[0].strip()
            self.yh = l[1].strip()

        self.statutes = ''
        l = self.etc.split('법령의 적용')
        if 1 < len(l):
            self.etc = l[0].strip()
            self.statutes = l[1].strip()

        self.facts = self.etc

        self.fy = f'{self.facts}\n{self.yh}'

        # self.etc = self.etc.replace('\n', ' ')
        # self.etc = self.etc.replace('에 대한 해당법조 및 형의 선택', '')
        # self.etc = self.etc.replace('에 대한 해당법조', '')
        # self.etc = self.etc.replace('형의 선택', '')
        # self.etc = self.etc.replace('피고인의 법정진술', '')
        # self.etc = self.etc.replace('[', '')
        # self.etc = self.etc.replace(']', '')
        # self.etc = self.etc.replace('1. ', '')
        # self.etc = self.etc.replace('2. ', '')
        # self.etc = self.etc.replace('3. ', '')
        # self.etc = self.etc.replace('4. ', '')
        # self.etc = self.etc.replace('범 죄 사 실', '')
        # self.etc = self.etc.replace('범죄전력', '')
        # self.etc = self.etc.replace('범죄사실', '')
        # self.etc = self.etc.replace('증거의 요지', '')
        # self.etc = self.etc.replace('법령의 적용', '')
        # self.etc = self.etc.replace('■', '')
        # self.etc = self.etc.strip()

        # self.ms = self.ms.replace('\n', ' ')
        # self.ms = self.ms.strip()

        self.raw = self.raw.strip()


        # self.etc = self.etc.replace('\n', ' ')
        # self.ms = self.ms.replace('\n', ' ')
        # self.raw = self.raw.replace('\n', ' ')
        # self.rsl = 'asf' # Reason of Sentence in law, 양형의 사유
        # self.statutes = 'df' # statutes, 법령

def getNumericInKorean(kStrNum):
    retval = 1
    retStr = kStrNum
    retStr = retStr.replace(',', '')
    if '만' in retStr:
        retval = retval * 10000
        retStr = retStr.replace('만', '')
    if '천' in retStr:
        retval = retval * 1000
        retStr = retStr.replace('천', '')
    retval = int(retStr) * retval
    return retval

def getMonthFromText(s):
    year  = re.findall(r'(\d+)년', s, re.S)
    y = 0 if len(year) == 0 else (int)(year[0]);
    month  = re.findall(r'(\d+)개?월', s, re.S)
    m = 0 if len(month) == 0 else (int)(month[0]);
    return y*12 + m

def generateDataSet(dataframe, max_cnt, classopt='class'):
    retval = pd.DataFrame({\
                           'id': pd.Series(dtype='int'), 'money': pd.Series(dtype='int'), 'prison': pd.Series(dtype='int'),'yooye': pd.Series(dtype='int'),\
                           'facts':[], 'statutes':[], 'yh':[], 'fy':[], 'raw':[]})
    cnts = {}
    for i, row in dataframe.iterrows():
        p = Precedent(row['precedent'])
        #print(f'p.ms: {p.ms}\n p.etc: {p.etc}')
        if '처한다' in p.ms and p.ms.startswith('피') and '및' not in p.ms:
            info = {}
            info['money'] = 0
            info['prison'] = 0
            info['yooye'] = 0
            # 주문\n피고인을 벌금 14,000,000(일천사백만)원에 처한다.\
            money  = re.findall(r'벌금\s+([^\s\(\)]+)\s*원', p.ms, re.S)
            if 0 < len(money): # 벌금형
                if classopt == 'class':
                    info['money'] = '0' if len(money) == 0 else getNumericInKorean(money[0])
                else:
                    info['money'] = '0' if len(money) == 0 else 1
            else: # 금고형, 집행유예 포함
                if '유예' in p.ms: # 집행유예 포함된 금고형.
                    msList = p.ms.split('처한다')
                    info['yooye'] = getMonthFromText(msList[1]) if classopt == 'class' else 1
                    info['prison'] = getMonthFromText(msList[0]) if classopt == 'class' else 1
                    # yy = 1
                    #prison = 1
                else: # 그냥 금고형
                    info['prison'] = getMonthFromText(p.ms) if classopt == 'class' else 1
                    #prison = 1

            if TARGET_START <= info[TARGET] and info[TARGET] <= TARGET_END:
            #    if prison in [0, 12, 18]:
                if info[TARGET] not in cnts:
                    cnts[info[TARGET]] = 0
                cnts[info[TARGET]] = cnts[info[TARGET]] + 1
                if cnts[info[TARGET]] > max_cnt:
                    continue
                retval.loc[i] = [i, [info['money']], [info['prison']], [info['yooye']], p.facts, p.statutes, p.yh, p.fy, p.raw]
            
            #retval.loc[i] = [i, [mn], [prison], [yy], p.etc, p.raw]  
    return retval

In [5]:
# # hs's rule
# [1,2,3,4,5]
# [6,10]
# []
# []
# []

# # yk's rule
# [1 2 3]
# [4 5 6]
# [7 8 9] 8
# [10 11 12]
# [13 14 15] 14
# [16 17 18]

# # 1~5 / 6~8 / 9~12 / 13~15 / 16~18 / 19~22 / 24~29 / 30~36 /

## 전처리된 데이터로 학습.
모델이 학습할 수 있도록 데이터의 형태를 변경해 학습합니다.

In [6]:
!pip install --quiet transformers==4.22.0
!pip install --quiet sentencepiece==0.1.97
!pip install --quiet pytorch-lightning==1.7.6
!pip install --quiet sklearn

In [7]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

from functools import reduce
import random

import torch
import numpy as np
import transformers
import pytorch_lightning as pl
from transformers import AutoTokenizer, BertForSequenceClassification

import pandas as pd

In [8]:
class StatutesDataModule(pl.LightningDataModule):
    def __init__(self, column, trainset, statutes, tokenizer, data, batch_size=16, max_input_len=512):
        super().__init__()
        self.column = column
        self.trainset = trainset
        self.statutes = statutes
        self.tokenizer = tokenizer
        self.data = data
        self.batch_size = batch_size
        self.max_input_len = max_input_len 
    
    def setup(self, stage):
        pass
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.data['train'], batch_size=self.batch_size, shuffle=True, collate_fn=self._collate_fn)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.data['valid'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.data['test'], batch_size=self.batch_size*2, shuffle=False, collate_fn=self._collate_fn)
    
    def _collate_fn(self, batch):
        inputs = self.tokenizer([x[self.trainset] for x in batch], max_length=self.max_input_len, padding=True, truncation=True, return_tensors='pt')
        labels = [x[self.column] for x in batch]

        return inputs, labels

In [9]:
class StatutesClassifier(pl.LightningModule):
    def __init__(self, statutes, backbone, learning_rate=1e-5, mode='multi-class', approx_val=2):
        super().__init__()
        self.statutes = statutes
        self.backbone = backbone
        self.learning_rate = learning_rate
        self.mode = mode
        self.criterion = torch.nn.CrossEntropyLoss() if mode == 'multi-class' else torch.nn.BCEWithLogitsLoss()
        self.approx_val = approx_val
        #self.criterion = torch.nn.MSELoss()
        #self.criterion = torch.nn.BCEWithLogitsLoss()
    
    def forward(self, batch):
        inputs, labels = batch
        logits = self.backbone(**inputs).logits
        targets = torch.zeros_like(logits)
        for i, label in enumerate(labels):
            label_id = [(self.statutes.index(x) if x in self.statutes else 0) for x in label]
            targets[i, label_id] = 1 / len(label_id) if self.mode == 'multi-class' else 1
        loss = self.criterion(logits, targets)

        return loss, logits

    def training_step(self, batch, batch_idx):
        loss, _ = self.forward(batch)
        return {'loss': loss}
    
    def validation_step(self, batch, batch_idx):
        return self._evaluation_step(batch) 
    
    def validation_epoch_end(self, outputs):
        self._evaluation_epoch_end(outputs)

    def test_step(self, batch, batch_idx):
        return self._evaluation_step(batch) 

    def test_epoch_end(self, outputs):
        self._evaluation_epoch_end(outputs, True)
   
    def _evaluation_step(self, batch):
        loss, logits = self.forward(batch)
        _, gts = batch
        if self.mode == 'multi-class':
            prs = np.array(self.statutes)[logits.argmax(-1).cpu()].tolist()
        else:
            prs = [np.array(self.statutes)[np.where(logit.sigmoid().cpu() > 0.5)[0]] for logit in logits]

        if isinstance(prs, list) == False:
            prs = [prs]
        
        return {'loss': loss.item(), 'gts': gts, 'prs': prs}

    def _evaluation_epoch_end(self, outputs, test=False):
        avg_loss = np.mean([x['loss'] for x in outputs])
        #print(outputs)
        gts = reduce(lambda x,y: x + y, [x['gts'] for x in outputs], [])
        prs = reduce(lambda x,y: x + y, [x['prs'] for x in outputs], [])
        # if self.mode == 'multi-class':
        #     #acc = sum([(pr in gt) for gt, pr in zip(gts, prs)]) / len(gts)
        #     acc = sum([abs(pr - gt[0] <= self.approx_val) for gt, pr in zip(gts, prs)]) / len(gts)
        # else:
        #     acc = sum([(set(pr) == set(gt)) for gt, pr in zip(gts, prs)]) / len(gts)
        
        #print(f'예측 형량: {prs}')
        self.answer = prs
            

    def configure_optimizers(self):
        grouped_params = [{'params': list(filter(lambda p: p.requires_grad, self.parameters())), 'lr': self.learning_rate}]
        optimizer = torch.optim.AdamW(grouped_params, lr=self.learning_rate)

        return {'optimizer': optimizer}

In [10]:
from sklearn.model_selection import train_test_split
import datasets
import tensorflow as tf
from datasets import Dataset

def convertDFtoDS(df):
    return tf.data.Dataset.from_tensor_slices(dict(df))

backbone_card = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(backbone_card)


In [11]:
# # coded by cat
# cfdxf,er?"
# ]\w
# ws"

In [12]:
import gc

def execTrain(dataset, target, trainset):
    #init cuda
    gc.collect()
    torch.cuda.empty_cache()

    #divide train/valid/test set
    train, test = train_test_split(dataset, test_size=0.2, random_state=1)
    train, valid = train_test_split(train, test_size=0.2, random_state=1)
    d = datasets.DatasetDict({'train': Dataset.from_dict(train), 'valid': Dataset.from_dict(valid), 'test': Dataset.from_dict(test)})

    # labels init
    labels = sorted(set.union(*map(set, dataset[target])))
    data_module = StatutesDataModule(target, trainset, labels, tokenizer, d, batch_size=16, max_input_len=512)

    # model
    backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(labels))
    model = StatutesClassifier(labels, backbone, learning_rate=1e-5, mode='multi-class', approx_val=TARGET_APPROX)

    # trainer
    n_gpus = 1 #torch.cuda.device_count()
    trainer = pl.Trainer(max_epochs=20, gpus=n_gpus, fast_dev_run=not True)

    # train and eval
    trainer.fit(model, data_module)
    trainer.test(model, data_module)

    return model

def initDataModule(dataset, target, trainset):
    #init cuda
    gc.collect()
    #torch.cuda.empty_cache()
    torch.no_grad()

    d = datasets.DatasetDict({'test': Dataset.from_dict(dataset)})

    # labels init
    labels = sorted(set.union(*map(set, dataset[target])))
    data_module = StatutesDataModule(target, trainset, labels, tokenizer, d, batch_size=16, max_input_len=512)

    return data_module

def execEval_bk(modelpath, data_module):

    # model
    #backbone = BertForSequenceClassification.from_pretrained(backbone_card, num_labels=len(labels))
    #model = StatutesClassifier(labels, backbone, learning_rate=1e-5, mode='multi-class')
    model = torch.load(modelpath)
    model.eval()

    # trainer
    #n_gpus = 1 #torch.cuda.device_count()
    #trainer = pl.Trainer(max_epochs=20, gpus=n_gpus, fast_dev_run=not True)

    #n_gpus = 1 #torch.cuda.device_count()
    trainer = pl.Trainer(max_epochs=20, fast_dev_run=not True, logger=None)

    trainer.test(model, data_module)
    return model.answer

def execEval(dataset, target, trainset, modelpath):
    #init cuda
    gc.collect()
    #torch.cuda.empty_cache()
    torch.no_grad()

    #divide train/valid/test set
    # train, test = train_test_split(dataset, test_size=0.2, random_state=1)
    # train, valid = train_test_split(train, test_size=0.2, random_state=1)
    #d = datasets.DatasetDict({'train': Dataset.from_dict(train), 'valid': Dataset.from_dict(valid), 'test': Dataset.from_dict(test)})
    d = datasets.DatasetDict({'test': Dataset.from_dict(dataset)})

    # labels init
    labels = sorted(set.union(*map(set, dataset[target])))
    data_module = StatutesDataModule(target, trainset, labels, tokenizer, d, batch_size=16, max_input_len=512)

    # model
    model = torch.load(modelpath)
    #model.eval()

    # trainer
    #n_gpus = 1 #torch.cuda.device_count()
    #trainer = pl.Trainer(max_epochs=20, gpus=n_gpus, fast_dev_run=not True)

    n_gpus = 1 #torch.cuda.device_count()
    trainer = pl.Trainer(max_epochs=1, fast_dev_run=True, logger=None)

    trainer.test(model, data_module)
    return model.answer

## RUN

In [13]:
import logging
import warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

# configure logging at the root level of Lightning
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)
warnings.filterwarnings(action='ignore')

In [14]:
class Judge:
    def __init__(self):
        # model. binary/class, prison/money/yooye
        self.bp = torch.load(f'{MODEL_DIR}/prison_binary_avail_fy_0_36_model.pt')
        self.bm = torch.load(f'{MODEL_DIR}/money_binary_avail_fy_0_100000000_model.pt')
        self.by = torch.load(f'{MODEL_DIR}/yooye_binary_avail_fy_0_36_model.pt')
        self.cp = torch.load(f'{MODEL_DIR}/prison_avail_fy_1_36_model.pt')
        self.cm = torch.load(f'{MODEL_DIR}/money_avail_fy_1_100000000_model.pt')
        self.cy = torch.load(f'{MODEL_DIR}/yooye_avail_fy_1_36_model.pt')

        self.trainer = pl.Trainer(max_epochs=1, fast_dev_run=True, enable_progress_bar=False)
        # n_gpus = 1 #torch.cuda.device_count()
        # self.trainer = pl.Trainer(max_epochs=1, gpus=n_gpus, fast_dev_run=True, enable_progress_bar=False)

    def genOneDataSet(self, facts, yh):
        retval = pd.DataFrame({\
                            'id': pd.Series(dtype='int'), 'money': pd.Series(dtype='int'), 'prison': pd.Series(dtype='int'),'yooye': pd.Series(dtype='int'),\
                            'facts':[], 'statutes':[], 'yh':[], 'fy':[], 'raw':[]})
        retval.loc[0] = [0, ['0'], ['0'], ['0'], facts, '', yh, f'{facts}\n{yh}', f'{facts}\n{yh}']
        return retval

    def testModel(self, model, trainer, data_module):
        trainer.test(model, data_module)
        return model.answer

    def judge(self, testfact, testyh):
        
        #init cuda
        gc.collect()
        torch.cuda.empty_cache()
        torch.no_grad()

        # generate dataset
        dataset = self.genOneDataSet(testfact, testyh)

        # divide train/valid/test set
        d = datasets.DatasetDict({'test': Dataset.from_dict(dataset)})

        # labels init
        labels = sorted(set.union(*map(set, dataset['prison'])))
        data_module = StatutesDataModule('prison', 'fy', labels, tokenizer, d, batch_size=16, max_input_len=512)

        # predict
        predBin = {}
        predCls = {}

        predBin['prison'] = self.testModel(self.bp, self.trainer, data_module)
        predBin['money'] = self.testModel(self.bm, self.trainer, data_module)
        predBin['yooye'] = self.testModel(self.by, self.trainer, data_module)

        predCls['prison'] = self.testModel(self.cp, self.trainer, data_module)
        predCls['money'] = self.testModel(self.cm, self.trainer, data_module)
        predCls['yooye'] = self.testModel(self.cy, self.trainer, data_module)

        # print results
        print('='*100)
        print(f'사실관계\n{testfact}\n양형의 이유\n{testyh}')
        print('징역[%d,%d월]' % (predBin['prison'][0], predCls['prison'][0]))
        print('유예[%d,%d월]' % (predBin['yooye'][0], predCls['yooye'][0]))
        print('벌금[%d,%d원]' % (predBin['money'][0], predCls['money'][0]))
        print('='*100)


In [15]:
j = None
j = Judge()

In [18]:
testyhs = []
testfact = '피고인은 2022. 4. 22. 19:10경 혈중알코올농도 0.038%의 술에 취한 상태로 부산 해운대구 B에 있는 C 앞 도로에서부터 부산 해운대구 D에 있는 E 앞 도로까지 약 1.8km의 구간에서 (차량번호 1 생략) 그랜저 승용차를 운전하였다.'
testyhs.append('')
testyhs.append('○ 불리한 정상: 음주운전으로 처벌받은 전력이 있음에도 또다시 이 사건 음주운전 범행을 저지른 점, ○ 유리한 정상: 피고인의 혈중알코올농도가 비교적 낮은 점, 자신의 범행을 시인하며 깊이 반성하는 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')
testyhs.append('○ 불리한 정상: 음주운전으로 처벌받은 전력이 있음에도 또다시 이 사건 음주운전 범행을 저지른 점, ○ 유리한 정상: 피고인의 혈중알코올농도가 비교적 낮은 점, 운전한 거리가 매우 멀지는 않은 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')
testyhs.append('○ 불리한 정상: 음주운전으로 처벌받은 전력이 있음에도 또다시 이 사건 음주운전 범행을 저지른 점, ○ 유리한 정상: 피고인의 혈중알코올농도가 비교적 낮은 점, 피고인의 음주 운전 벌금 전력으로부터 상당한 시간적 간격이 있는 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')
testyhs.append('○ 불리한 정상: 음주운전으로 처벌받은 전력이 있음에도 또다시 이 사건 음주운전 범행을 저지른 점, ○ 유리한 정상: 피고인의 혈중알코올농도가 비교적 낮은 점, 월세를 얻어 두 자녀를 양육하고 있는 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')
testyhs.append('○ 불리한 정상: 음주운전으로 처벌받은 전력이 있음에도 또다시 이 사건 음주운전 범행을 저지른 점, ○ 유리한 정상: 피고인의 혈중알코올농도가 비교적 낮은 점, 월세를 얻어 두 자녀를 양육하고 있는 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')
testyhs.append('○ 불리한 정상: 음주운전으로 처벌받은 전력이 있음에도 또다시 이 사건 음주운전 범행을 저지른 점, ○ 유리한 정상: 피고인의 혈중알코올농도가 비교적 낮은 점, 피고인이 장애를 가지고 있는 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')
testyhs.append('○ 불리한 정상: 음주운전으로 처벌받은 전력이 있음에도 또다시 이 사건 음주운전 범행을 저지른 점, ○ 유리한 정상: 피고인의 혈중알코올농도가 비교적 낮은 점, 피고인이 장애를 가지고 있는 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')

for testyh in testyhs:
    j.judge(testfact, testyh)

사실관계
피고인은 2022. 4. 22. 19:10경 혈중알코올농도 0.038%의 술에 취한 상태로 부산 해운대구 B에 있는 C 앞 도로에서부터 부산 해운대구 D에 있는 E 앞 도로까지 약 1.8km의 구간에서 (차량번호 1 생략) 그랜저 승용차를 운전하였다.
양형의 이유￦n
징역[1,18월]
유예[1,24월]
벌금[0,5000000원]
사실관계
피고인은 2022. 4. 22. 19:10경 혈중알코올농도 0.038%의 술에 취한 상태로 부산 해운대구 B에 있는 C 앞 도로에서부터 부산 해운대구 D에 있는 E 앞 도로까지 약 1.8km의 구간에서 (차량번호 1 생략) 그랜저 승용차를 운전하였다.
양형의 이유￦n○ 불리한 정상: 음주운전으로 처벌받은 전력이 있음에도 또다시 이 사건 음주운전 범행을 저지른 점, ○ 유리한 정상: 피고인의 혈중알코올농도가 비교적 낮은 점, 자신의 범행을 시인하며 깊이 반성하는 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.
징역[1,18월]
유예[0,24월]
벌금[0,5000000원]
사실관계
피고인은 2022. 4. 22. 19:10경 혈중알코올농도 0.038%의 술에 취한 상태로 부산 해운대구 B에 있는 C 앞 도로에서부터 부산 해운대구 D에 있는 E 앞 도로까지 약 1.8km의 구간에서 (차량번호 1 생략) 그랜저 승용차를 운전하였다.
양형의 이유￦n○ 불리한 정상: 음주운전으로 처벌받은 전력이 있음에도 또다시 이 사건 음주운전 범행을 저지른 점, ○ 유리한 정상: 피고인의 혈중알코올농도가 비교적 낮은 점, 운전한 거리가 매우 멀지는 않은 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.
징역[1,18월]
유예[0,24월]
벌금[0,5000000원]
사실관계
피고인은 2022. 4. 22

In [19]:
testyhs = []
testfact = '피고인은 2022. 3. 31. 23:38경 서울 마포구 B 인근 도로에서부터 서울 금천구 C 앞 도로에 이르기까지 약 12km 구간에서 혈중알코올농도 0.118%의 술에 취한 상태로 (차량번호 1 생략) 스파크 승용차를 운전하였다.'
testyhs.append('')
testyhs.append('○ 불리한 정상: 혈중알코올농도 수치가 높고 음주상태에서 운전한 거리가 상당한 점, ○ 유리한 정상: 자신의 범행을 시인하며 깊이 반성하는 점, 피고인이 아무런 범죄 전력이 없는 초범인 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')
testyhs.append('○ 불리한 정상: 혈중알코올농도 수치가 높고 음주상태에서 운전한 거리가 상당한 점, ○ 유리한 정상: 자신의 범행을 시인하며 깊이 반성하는 점, 피고인이 아무런 범죄 전력이 없는 초범인 점, 월세를 얻어 두 자녀를 양육하고 있는 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')
testyhs.append('○ 불리한 정상: 혈중알코올농도 수치가 높고 음주상태에서 운전한 거리가 상당한 점, ○ 유리한 정상: 자신의 범행을 시인하며 깊이 반성하는 점, 피고인이 아무런 범죄 전력이 없는 초범인 점, 피고인이 차량을 처분하면서 재범하지 않을 것을 다짐하고 있는 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')
testyhs.append('○ 불리한 정상: 혈중알코올농도 수치가 높고 음주상태에서 운전한 거리가 상당한 점, ○ 유리한 정상: 자신의 범행을 시인하며 깊이 반성하는 점, 피고인이 아무런 범죄 전력이 없는 초범인 점, 피고인이 장애를 가지고 있는 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')
testyhs.append('○ 불리한 정상: 혈중알코올농도 수치가 높고 음주상태에서 운전한 거리가 상당한 점, ○ 유리한 정상: 자신의 범행을 시인하며 깊이 반성하는 점, 피고인이 아무런 범죄 전력이 없는 초범인 점, 대리운전 업체에 전화하여 상당시간을 기다리다 범행에 이른 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')
testyhs.append('○ 불리한 정상: 혈중알코올농도 수치가 높고 음주상태에서 운전한 거리가 상당한 점, ○ 유리한 정상: 자신의 범행을 시인하며 깊이 반성하는 점, 피고인이 아무런 범죄 전력이 없는 초범인 점, 피고인이 고령인 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.')

for testyh in testyhs:
    j.judge(testfact, testyh)

사실관계
피고인은 2022. 3. 31. 23:38경 서울 마포구 B 인근 도로에서부터 서울 금천구 C 앞 도로에 이르기까지 약 12km 구간에서 혈중알코올농도 0.118%의 술에 취한 상태로 (차량번호 1 생략) 스파크 승용차를 운전하였다.
양형의 이유￦n
징역[1,18월]
유예[0,24월]
벌금[0,5000000원]
사실관계
피고인은 2022. 3. 31. 23:38경 서울 마포구 B 인근 도로에서부터 서울 금천구 C 앞 도로에 이르기까지 약 12km 구간에서 혈중알코올농도 0.118%의 술에 취한 상태로 (차량번호 1 생략) 스파크 승용차를 운전하였다.
양형의 이유￦n○ 불리한 정상: 혈중알코올농도 수치가 높고 음주상태에서 운전한 거리가 상당한 점, ○ 유리한 정상: 자신의 범행을 시인하며 깊이 반성하는 점, 피고인이 아무런 범죄 전력이 없는 초범인 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.
징역[1,8월]
유예[0,24월]
벌금[0,5000000원]
사실관계
피고인은 2022. 3. 31. 23:38경 서울 마포구 B 인근 도로에서부터 서울 금천구 C 앞 도로에 이르기까지 약 12km 구간에서 혈중알코올농도 0.118%의 술에 취한 상태로 (차량번호 1 생략) 스파크 승용차를 운전하였다.
양형의 이유￦n○ 불리한 정상: 혈중알코올농도 수치가 높고 음주상태에서 운전한 거리가 상당한 점, ○ 유리한 정상: 자신의 범행을 시인하며 깊이 반성하는 점, 피고인이 아무런 범죄 전력이 없는 초범인 점, 월세를 얻어 두 자녀를 양육하고 있는 점, 위와 같은 정상들과 그밖에 피고인의 연령, 성행과 환경, 범행의 동기 및 경위, 범행 후의 정황 등 변론에 나타난 모든 양형조건을 종합하여 주문과 같이 형을 정한다.
징역[1,8월]
유예[0,24월]
벌금[0,5000000원]
사실관계
피고인은 2022. 3. 31. 23:38경 서울 마포구 B 인